In [ ]:
## NN Dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

## Other Dependencies
import pandas as pd
from matplotlib import pyplot as plt

## Initial Data import and pre-processing for Neural Network model

In [ ]:
## Import dataset csv

churn_df = pd.read_csv('Resources/clean_churn_db.csv')
attrition_df = pd.read_csv('Resources/BankChurners.csv', usecols=['Attrition_Flag'])
churn_df

In [ ]:
print(f"The amount of Attrited Customers/Existing Customers in the dataset is 1628/8500 or {round(1627/8500 * 100,2)}%")

In [ ]:
for column in churn_df.columns:
    print(column)

In [ ]:
## Define feature values X
X = churn_df.values

## Define target values y
y_df = attrition_df.replace({'Existing Customer':0, 'Attrited Customer':1}).copy()
y = y_df['Attrition_Flag']

In [ ]:
## Split into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 7)

In [ ]:
## Instance Scaler
scaler = StandardScaler()

## Fit Scaler
X_scaler = scaler.fit(X_train)

## Scale Data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
len(X_train_scaled[0])

## Initial Modelling Attempt
Using all the availible features in the dataset

In [ ]:
## Layering, beginning with 1 hidden layer
input_features = len(X_train[0])
hidden_layer_1 = 25
outputs = 1

nn = tf.keras.models.Sequential(name='initial')

## First Hidden Layer + Input
nn.add(tf.keras.layers.Dense(units = hidden_layer_1, input_dim = input_features, activation = 'relu'))

##Output Layer
nn.add(tf.keras.layers.Dense(units = outputs, activation='sigmoid'))

nn.summary()

In [ ]:
## Compile and fit
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',])
initial_model = nn.fit(X_train_scaled, y_train, validation_data = (X_test_scaled, y_test) , epochs=100)

In [ ]:
initial_loss, initial_accuracy = nn.evaluate(X_test_scaled, y_test, verbose = 2)
print(f"Loss:{initial_loss}, Accuracy: {initial_accuracy}")

In [ ]:
initial_loss, initial_accuracy = nn.evaluate(X_test_scaled, y_test, verbose = 2)
print(f"Loss:{initial_loss}, Accuracy: {initial_accuracy}")

In [ ]:
## What does this 93% accuracy mean?

In [ ]:
plt.plot(initial_model.history['accuracy'])
plt.plot(initial_model.history['val_accuracy'])
plt.title("Initial Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(['Training','Validation'], loc = 'center')
plt.show()

In [ ]:
plt.plot(initial_model.history['loss'])
plt.plot(initial_model.history['val_loss'])
plt.title("Initial Model Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(['Training','Validation'], loc = 'center')
plt.show()

Model seems to be overfitted, adjustments to follow.

## Adjusting Model using L2 regularization

In [ ]:
from tensorflow.keras import regularizers

In [ ]:
## Layering, beginning with 1 hidden layer
input_features = len(X_train[0])
hidden_layer_1 = 25
outputs = 1

nn = tf.keras.models.Sequential(name='l2_reg')

## First Hidden Layer + Input
nn.add(tf.keras.layers.Dense(units = hidden_layer_1, input_dim = input_features, activation = 'relu',
                            kernel_regularizer=regularizers.l2(0.001)))

##Output Layer
nn.add(tf.keras.layers.Dense(units = outputs, activation='sigmoid'))

nn.summary()

In [ ]:
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',])
l1_reg = nn.fit(X_train_scaled, y_train, validation_data = (X_test_scaled, y_test) , epochs=100)

In [ ]:
plt.plot(l1_reg.history['accuracy'])
plt.plot(l1_reg.history['val_accuracy'])
plt.title("L2 Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(['Training','Validation'], loc = 'center')
plt.show()

In [ ]:
plt.plot(l1_reg.history['loss'])
plt.plot(l1_reg.history['val_loss'])
plt.title("L2 Model Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(['Training','Validation'], loc = 'center')
plt.show()

In [ ]:
## Looks good at first, but the model still finds out a pattern early and does not adapt to new information

## Adjusting model using Dropout

In [ ]:
## Layering, beginning with 1 hidden layer
input_features = len(X_train[0])
hidden_layer_1 = 25
outputs = 1

nn = tf.keras.models.Sequential(name='dropout')

## First Hidden Layer + Input

nn.add(tf.keras.layers.Dense(units = hidden_layer_1, input_dim = input_features, activation = 'relu'))
nn.add(tf.keras.layers.Dropout(.2))

##Output Layer
nn.add(tf.keras.layers.Dense(units = outputs, activation='sigmoid'))


nn.summary()

In [ ]:
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',])
dropout_model = nn.fit(X_train_scaled, y_train, validation_data = (X_test_scaled, y_test) , epochs=100)

In [ ]:
plt.plot(dropout_model.history['accuracy'])
plt.plot(dropout_model.history['val_accuracy'])
plt.title("Dropout Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(['Training','Validation'], loc = 'center')
plt.show()

In [ ]:
plt.plot(dropout_model.history['loss'])
plt.plot(dropout_model.history['val_loss'])
plt.title("Dropout Model Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(['Training','Validation'], loc = 'center')
plt.show()

## Using both L2 and Dropout

In [ ]:
## Layering, beginning with 1 hidden layer
input_features = len(X_train[0])
hidden_layer_1 = 25
outputs = 1

nn = tf.keras.models.Sequential(name='dropout_and_l2')

## First Hidden Layer + Input
nn.add(tf.keras.layers.Dense(units = hidden_layer_1, input_dim = input_features, activation = 'relu',
                             kernel_regularizer=regularizers.l2(0.001)))
nn.add(tf.keras.layers.Dropout(.2))


##Output Layer
nn.add(tf.keras.layers.Dense(units = outputs, activation='sigmoid'))


nn.summary()

In [ ]:
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 
                                                                  tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
dropout_l2_model = nn.fit(X_train_scaled, y_train, validation_data = (X_test_scaled, y_test) , epochs=100)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 8))

axs[0,0].plot(dropout_l2_model.history['loss'])
axs[0,0].plot(dropout_l2_model.history['val_loss'])
axs[0,0].set_title("Dropout/L2 Loss")

axs[1,0].plot(dropout_l2_model.history['accuracy'])
axs[1,0].plot(dropout_l2_model.history['val_accuracy'])
axs[1,0].set_title("Dropout/L2 Accuracy")

axs[0,1].plot(dropout_l2_model.history['recall'])
axs[0,1].plot(dropout_l2_model.history['val_recall'])
axs[0,1].set_title("Dropout/L2 Recall")

axs[1,1].plot(dropout_l2_model.history['precision'])
axs[1,1].plot(dropout_l2_model.history['val_precision'])
axs[1,1].set_title("Dropout/L2 Precision")

fig.tight_layout()